<a href="https://colab.research.google.com/github/kaylaque/senyumin/blob/main/Transfer_Learning_VGG19_FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
# directory
dir = '/content/drive/MyDrive/dsc/PLAN A [DATASET FAILED]/fer2013/fer'
filepath = '/content/drive/MyDrive/dsc/PLAN B [ONGOING]/'

In [ ]:
import cv2
img = cv2.imread(dir + '/angry/img_0.png')
img.shape

(224, 224, 3)

In [ ]:
# augmentate data with ImageDataGenerator and amount of validation's data is 40%  of total data
aug_data = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=5,
    height_shift_range=0.2,
    # zca_whitening=True,
    vertical_flip = True,
    brightness_range=[0.2,1.0],
    zoom_range=[1.5,1.0],
    fill_mode='nearest',
    validation_split=0.4
)

In [ ]:
# define training data
train_gen = aug_data.flow_from_directory(
    dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)
# define validation data
val_gen = aug_data.flow_from_directory(
    dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

Found 4739 images belonging to 5 classes.
Found 3156 images belonging to 5 classes.


In [ ]:
base_model = keras.applications.VGG19(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(inputs)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(5, activation = 'softmax')(x)
model = keras.Model(inputs, outputs)

model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
______________________________________________________________

In [ ]:
callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
    options=None
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

In [ ]:
epochs = 20
batch_size = 32
hist = model.fit(train_gen,
                 batch_size = 32,
                 steps_per_epoch=len(train_gen)//batch_size, 
                 epochs=epochs, 
                #  callbacks=callbacks, 
                #  validation_steps=,
                 validation_data=val_gen,
                 verbose=1)

Epoch 1/20
4/4 [==============================] - 1004s 329s/step - loss: 1.5595 - categorical_accuracy: 0.2865 - val_loss: 1.4695 - val_categorical_accuracy: 0.3663
Epoch 2/20
4/4 [==============================] - 914s 299s/step - loss: 1.5197 - categorical_accuracy: 0.2781 - val_loss: 1.4424 - val_categorical_accuracy: 0.3663
Epoch 3/20
4/4 [==============================] - ETA: 0s - loss: 1.4767 - categorical_accuracy: 0.3438

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 10

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
normalization_1 (Normalizati (None, 150, 150, 3)       7         
_________________________________________________________________
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565

In [ ]:
history = model.fit(train_gen,
                 batch_size = 32,
                 steps_per_epoch=len(train_gen)//batch_size, 
                 epochs=epochs, 
                #  callbacks=callbacks, 
                #  validation_steps=,
                 validation_data=val_gen,
                 verbose=1)

Epoch 1/5
4/4 [==============================] - 1020s 326s/step - loss: 1.4011 - categorical_accuracy: 0.3490 - val_loss: 1.4259 - val_categorical_accuracy: 0.3663
Epoch 2/5
4/4 [==============================] - 1022s 328s/step - loss: 1.4635 - categorical_accuracy: 0.2802 - val_loss: 1.4313 - val_categorical_accuracy: 0.3663
Epoch 3/5
4/4 [==============================] - 1016s 326s/step - loss: 1.4601 - categorical_accuracy: 0.3125 - val_loss: 1.4313 - val_categorical_accuracy: 0.3663
Epoch 4/5
4/4 [==============================] - 1017s 327s/step - loss: 1.4345 - categorical_accuracy: 0.2917 - val_loss: 1.4190 - val_categorical_accuracy: 0.3663
Epoch 5/5
4/4 [==============================] - 1011s 325s/step - loss: 1.3903 - categorical_accuracy: 0.3063 - val_loss: 1.4226 - val_categorical_accuracy: 0.3663


In [ ]:
%cd "/content/drive/MyDrive/dsc/PLAN B [ONGOING]/"
model.save_weights('model_weightsVGG.h5')
model.save('model_keras30.h5')
print("Saved model to disk")

/content/drive/MyDrive/dsc/PLAN B [ONGOING]
Saved model to disk
